<a href="https://colab.research.google.com/github/FNS-Division/GIGA-mapping-training/blob/main/get_ookla_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get open data from Ookla Speedtest on mobile and fixed network performance

In [38]:
!pip install s3fs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 76.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==1

In [39]:
import pandas as pd
import pyarrow.parquet as pq
import os
from pathlib import Path
import ipywidgets as widgets
import s3fs

In [40]:
# @title Configure Ookla download
# Data Selection Parameters
data_params = {
    'data_type': widgets.Dropdown(
        options=['fixed', 'mobile'],
        value='fixed',
        description='Data Type:',
        style={'description_width': 'initial'}
    ),
    'year': widgets.IntText(
        value=2024,
        description='Year:',
        style={'description_width': 'initial'}
    ),
    'quarter': widgets.IntSlider(
        value=2,
        min=1,
        max=4,
        step=1,
        description='Quarter:',
        style={'description_width': 'initial'}
    )
}

# Coordinate Bounds
coordinate_bounds = {
    'min_x': widgets.FloatText(
        value=16.991831,
        description='Min X:',
        style={'description_width': 'initial'}
    ),
    'max_x': widgets.FloatText(
        value=17.151596,
        description='Max X:',
        style={'description_width': 'initial'}
    ),
    'min_y': widgets.FloatText(
        value=-22.657422,
        description='Min Y:',
        style={'description_width': 'initial'}
    ),
    'max_y': widgets.FloatText(
        value=-22.414187,
        description='Max Y:',
        style={'description_width': 'initial'}
    )
}

# Create layout for data parameters
data_layout = widgets.VBox([
    widgets.HTML(value='<b>Data Selection Parameters</b>'),
    data_params['data_type'],
    data_params['year'],
    data_params['quarter']
])

# Create layout for coordinate bounds
coordinates_layout = widgets.VBox([
    widgets.HTML(value='<b>Coordinate Bounds</b>'),
    coordinate_bounds['min_x'],
    coordinate_bounds['max_x'],
    coordinate_bounds['min_y'],
    coordinate_bounds['max_y']
])

# Combine both layouts
combined_layout = widgets.VBox([
    data_layout,
    widgets.HTML(value='<br>'),  # Add some spacing
    coordinates_layout
])

display(combined_layout)


In [41]:
# @title Helper functions
def create_output_directory(base_path="ookla_data"):
    """
    Create a directory to store the output files if it doesn't exist.

    Args:
        base_path (str): The path where you want to create the directory

    Returns:
        Path: Path object pointing to the created directory
    """
    output_dir = Path(base_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    return output_dir


def get_perf_tiles_parquet_url(service: str, year: int, quarter: int) -> str:
    """
    Generate the correct S3 URL for Ookla performance tiles data.

    Args:
        service (str): Either 'fixed' or 'mobile'
        year (int): Year of the data
        quarter (int): Quarter number (1-4)
    """
    quarter_start = f"{year}-{(((quarter - 1) * 3) + 1):02}-01"
    url = f"s3://ookla-open-data/parquet/performance/type={service}/year={year}/quarter={quarter}/{quarter_start}_performance_{service}_tiles.parquet"
    return url

def download_ookla_data(data_type, year, quarter, bounds, output_path):
    """
    Download and filter Ookla network performance data using efficient Parquet filtering.

    Args:
        data_type (str): Either 'fixed' or 'mobile'
        year (int): Year of the data
        quarter (int): Quarter of the data (1-4)
        bounds (dict): Dictionary containing geographical bounds
        output_path (Path): Path to save the output CSV
    """
    # Get the correct URL
    url = get_perf_tiles_parquet_url(data_type, year, quarter)

    # Create bbox filters for efficient Parquet reading
    bbox_filters = [
        ('tile_y', '<=', bounds['max_y']),
        ('tile_y', '>=', bounds['min_y']),
        ('tile_x', '<=', bounds['max_x']),
        ('tile_x', '>=', bounds['min_x'])
    ]

    # Columns to retrieve
    columns = ['tile_x', 'tile_y', 'tests', 'avg_d_kbps', 'avg_lat_ms']

    print(f"Downloading {data_type} data from {url}")
    print(f"Applying bounds filter: {bounds}")

    # Read parquet file with predicate pushdown filtering
    try:
        df = pd.read_parquet(
            url,
            filters=bbox_filters,
            columns=columns,
            storage_options={"s3": {"anon": True}}
        )

        # Save to CSV
        output_file = output_path / f"{data_type}_network_data.csv"
        df.to_csv(output_file, index=False)
        print(f"Saved filtered data to {output_file}")
        print(f"Retrieved {len(df)} records")

        return df

    except Exception as e:
        print(f"Error downloading data: {str(e)}")
        raise

In [43]:
# Dictionary of bounds
bounds = {
    'min_x': coordinate_bounds["min_x"].value,
    'max_x': coordinate_bounds["max_x"].value,
    'min_y': coordinate_bounds["min_y"].value,
    'max_y': coordinate_bounds["max_y"].value
}

# Create output directory
output_dir = create_output_directory()

# Download mobile data
download_ookla_data(
    data_type="mobile",
    year=data_params["year"].value,
    quarter=data_params["quarter"].value,
    bounds=bounds,
    output_path=output_dir
)

# Download fixed data
download_ookla_data(
    data_type="fixed",
    year=data_params["year"].value,
    quarter=data_params["quarter"].value,
    bounds=bounds,
    output_path=output_dir
)

Applying bounds filter: {'min_x': 16.991831, 'max_x': 17.151596, 'min_y': -22.657422, 'max_y': -22.414187}
Saved filtered data to ookla_data/fixed_network_data.csv
Retrieved 334 records


,tile_x,tile_y,tests,avg_d_kbps,avg_lat_ms
0,17.0535,-22.4186,2,13131,10
1,17.0590,-22.4186,7,12492,15
2,17.0645,-22.4186,2,3973,11
3,17.0645,-22.4237,1,1719,43
4,17.0700,-22.4288,6,110512,4
...,...,...,...,...,...
329,17.0975,-22.6267,34,26346,15
330,17.1030,-22.6267,2,9157,10
331,17.0590,-22.6520,2,134817,3
332,17.0755,-22.6368,4,17098,50


## Unzip OpenCellID files

In [ ]:
import geopandas as gpd
import gzip
import shutil

In [2]:
def unzip_gz(gz_file_path, output_path=None):
    """
    Unzips a .gz file and returns the path to the unzipped file.

    Args:
        gz_file_path (str): Path to the .gz file
        output_path (str): Path for the output file.

    Returns:
        str: Path to the unzipped file
    """
    # Create output directory if it doesn't exist
    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Unzip the file
    with gzip.open(gz_file_path, 'rb') as f_in:
        with open(output_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    return output_path

In [ ]:
location_of_gz_file = "/content/626.csv.gz"
unzipped_cell_sites = unzip_gz(location_of_gz_file,"/content/cell_sites.csv")
cell_sites = pd.read_csv(unzipped_cell_sites, header=None,
                        names = ['radio', 'mcc', 'net', 'area', 'cell', 'unit', 'lon', 'lat',
                                'range', 'samples', 'changeable', 'created', 'updated', 'averageSignal'])
cell_sites_gdf = gpd.GeoDataFrame(cell_sites, geometry=gpd.points_from_xy(cell_sites.lon, cell_sites.lat), crs="EPSG:4326").drop(columns=["lon", "lat"])

In [ ]:
cell_sites_gdf.to_file("/content/cell_sites.geojson", driver='GeoJSON')